<a href="https://colab.research.google.com/github/FelipeTbs/Linguagens-Formais---TAES---CIn-UFPE/blob/main/TAES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importações de dependências

In [ ]:
import google.generativeai as genai
from openai import OpenAI
import pandas as pd
import time
from difflib import SequenceMatcher
from google.colab import drive
from google.colab import userdata
import os
import glob
import time
from tqdm.notebook import tqdm

!pip install -q -U google-generativeai openai "rich<14"
!pip install -q anthropic
print("Bibliotecas instaladas!")

##Artefatos de Engenharia de prompt

In [ ]:


PROMPT_SPECIALIST_SYSTEM = """
A seguir haverá uma especificação formal escrita em Alloy. Sua tarefa é interpretar esse
modelo e explicá-lo de forma totalmente compreensível para um leitor leigo, mantendo
precisão conceitual. Produza a saída em três seções: (1) Resumo Executivo, com 2 a 3
frases em linguagem simples e sem jargões, explicando o propósito do modelo e o
problema do mundo real que ele representa; (2) Explicação Detalhada da Especificação
Formal, descrevendo cada parte do modelo em ordem lógica, explicando o significado de
assinaturas, relações, predicados, cardinalidades (como lone, one, some, set),
restrições e estrutura semântica. Nesta seção, apresente explicações narrativas
passo a passo, exemplos de entradas e saídas esperadas, possíveis interpretações,
tabelas estruturadas quando útil, e traduções conceituais para linguagem natural;
(3) Pontos Críticos, Restrições e Alertas, listando limitações, ambiguidades,
pressupostos implícitos e restrições importantes derivadas do modelo.

Regras obrigatórias: utilizar linguagem acessível para alguém não técnico
(por exemplo, um analista de negócios), evitar jargões e, quando usados,
explicar imediatamente; não apenas traduzir o código, mas interpretar seu
significado e finalidade; fornecer contexto prático e clareza sem precisar
conhecimento prévio em lógica ou formal methods; organizar a saída de forma
clara e legível.

O formato final da resposta deve seguir exatamente:
	1.	Resumo Executivo
	2.	Explicação Detalhada da Especificação Formal
	3.	Pontos Críticos, Restrições e Alertas

Agora, aqui está o modelo a ser analisado:
"""

##Importação de datasets

In [ ]:
import anthropic

drive.mount('/content/drive')

CAMINHO_PASTA = '/content/drive/MyDrive/dataset'

def carregar_dataset_filtrado(caminho, limite=5, buscar_nomes=None):
    """
    Carrega apenas 'limite' arquivos.
    Se 'buscar_nomes' for passado, tenta achar esses arquivos primeiro.
    """
    print(f"Lendo pasta: {caminho}")
    todos_arquivos = glob.glob(f"{caminho}/*.als")

    if len(todos_arquivos) == 0:
        print("Nenhum arquivo encontrado! Verifique o caminho.")
        return pd.DataFrame()

    arquivos_selecionados = []

    if buscar_nomes:
        for nome_buscado in buscar_nomes:
            match = next((f for f in todos_arquivos if nome_buscado in os.path.basename(f)), None)
            if match:
                arquivos_selecionados.append(match)
                print(f"   --> Encontrado prioritário: {os.path.basename(match)}")

    for arq in todos_arquivos:
        if len(arquivos_selecionados) >= limite:
            break
        if arq not in arquivos_selecionados:
            arquivos_selecionados.append(arq)

    dados = []
    print(f"\n Importando {len(arquivos_selecionados)} arquivos selecionados:")

    for arq in arquivos_selecionados:
        try:
            with open(arq, 'r', encoding='utf-8') as f:
                conteudo = f.read()
            nome = os.path.basename(arq)
            print(f"   - {nome}")

            dados.append({
                "arquivo": nome,
                "codigo_alloy": conteudo,
                "tamanho": len(conteudo)
            })
        except Exception as e:
            print(f"Erro ao ler {arq}: {e}")

    return pd.DataFrame(dados)


PRIORIDADES = ['grandpa2.als', 'genealogy.als', 'addressBook.als', 'filesystem.als', 'ringElection2.als']

if os.path.exists('/content/drive'):
    df_alloy = carregar_dataset_filtrado(CAMINHO_PASTA, limite=5, buscar_nomes=PRIORIDADES)
    display(df_alloy)
else:
    print("Rode a célula para conectar o Drive!")


##Importação de Modelos

In [ ]:


try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("Chaves recuperadas dos Secrets com sucesso!")
except Exception as e:
    print(f"Erro ao pegar chaves: {e}")

# Configuração dos modelos
genai.configure(api_key=GOOGLE_API_KEY)
client_gpt = OpenAI(api_key=OPENAI_API_KEY)
print('Chaves configuradas com sucesso!')

ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

client_claude = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

def generalist_model_claude(alloy_code):
  try:
      final_prompt = f"{PROMPT_SPECIALIST_SYSTEM}\n\n{alloy_code}"
      message = client_claude.messages.create(
            model="claude-sonnet-4-5-20250929", # O modelo SOTA atual
            max_tokens=2000,
            temperature=0.1, # Baixa temperatura para precisão
            messages=[
                {"role": "user", "content": final_prompt}
            ]
      )
      return message.content[0].text

  except Exception as e:
        return f"Erro Claude: {e}"

def generalist_model(alloy_code):
    """GPT-4o (Zero-Shot)"""
    try:
        final_prompt = f"Explique em Português do Brasil o que esse código faz, lembrem que eu não tenho conhecimento em programação e lógica:\n\n{alloy_code}"
        response = client_gpt.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": final_prompt}
            ],
            temperature=0.1 # Baixa temperatura para precisão
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Erro Gemini: {e}"

def specialist_model(alloy_code):
    """GPT-4o (Few-Shot + System Prompt)"""
    try:
        response = client_gpt.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": PROMPT_SPECIALIST_SYSTEM}, #manual de alloy aqui (definir especialidade)
                {"role": "user", "content": alloy_code}
            ],
            temperature=0.1 # Baixa temperatura para precisão
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Erro GPT: {e}"

##Comparação e Resultados

In [ ]:

# Lista para guardar os dados
resultados_finais = []

print(f"Iniciando processamento de {len(df_alloy)} arquivos...")

for index, row in tqdm(df_alloy.iterrows(), total=df_alloy.shape[0]):
    nome_arquivo = row['arquivo']
    codigo = row['codigo_alloy']

    # --- 1. MODELO GENERALISTA (GPT-4o Zero-Shot) ---
    start_gen = time.time()
    resp_gen = generalist_model(codigo)
    tempo_gen = time.time() - start_gen

    # --- 2. MODELO ESPECIALISTA (GPT-4o + System Prompt) ---
    start_esp = time.time()
    resp_esp = specialist_model(codigo)
    tempo_esp = time.time() - start_esp

    # (Opcional) --- 3. CLAUDE (Se estiver funcionando) ---
    # start_claude = time.time()
    # resp_claude = generalist_model_claude(codigo)
    # tempo_claude = time.time() - start_claude

    # --- SALVAR RESULTADOS ---
    resultados_finais.append({
        "Arquivo": nome_arquivo,
        "Tamanho_Codigo": len(codigo),

        # Dados do Generalista
        "Generalista_Output": resp_gen,
        "Generalista_Tempo_Sec": round(tempo_gen, 2),
        "Generalista_Tamanho_Resp": len(resp_gen),

        # Dados do Especialista
        "Especialista_Output": resp_esp,
        "Especialista_Tempo_Sec": round(tempo_esp, 2),
        "Especialista_Tamanho_Resp": len(resp_esp)

    })

    # Pausa de 1 segundo para não estourar limite da OpenAI (Rate Limit)
    time.sleep(1)

# --- EXPORTAÇÃO FINAL ---
df_resultados = pd.DataFrame(resultados_finais)

# Salva em arquivos reais
df_resultados.to_excel("Resultados_Completos_Alloy.xlsx", index=False)
df_resultados.to_csv("Resultados_Completos_Alloy.csv", index=False)

print("\n Processamento Concluído!")
print("Os arquivos 'Resultados_Completos_Alloy' (.xlsx e .csv) foram salvos na aba de arquivos à esquerda.")

display(df_resultados.head())

In [ ]:


JARGOES_TECNICOS = [
    # Palavras reservadas do Alloy (se aparecerem, é erro grave de tradução)
    "sig ", "fact ", "pred ", "assert ", "check ", "run ", "fun ",
    "abstract", "extends", "univ", "iden", "none", "this",
    "one", "lone", "some", "all", " set ", "disj", "let ", "iff",
    "util/", "ordering",

    # Traduções literais que soam matemáticas demais
    "assinatura", "predicado", "axioma", "multiplicidade",
    "instância", "tupla", "produto cartesiano", "fecho transitivo",
    "bijective", "injetora", "sobrejetora", "succ", "prevs"
]

# Lista EXPANDIDA de Negócios (Recompensa)
# Foca em verbos de ação, consequências e termos de domínio
TERMOS_DE_NEGOCIO = [
    # Substantivos de Abstração
    "entidade", "objeto", "elemento", "item", "indivíduo", "pessoa",
    "usuário", "sistema", "modelo", "estrutura", "cenário", "requisito",

    # Verbos e Termos de Regra/Consequência (O mais importante!)
    "regra", "restrição", "garante", "assegura", "impede", "evita",
    "permite", "proibido", "obrigatório", "necessário", "possível",
    "válido", "inválido", "consistente", "inconsistente",

    # Relações Humanas (Para os datasets específicos como Grandpa/AddressBook)
    "pai", "mãe", "filho", "avô", "ancestral", "descendente",
    "contém", "pertence", "conexão", "vínculo",

    # Domínio: Sistemas Distribuídos (RingElection)
    "processo", "nó", "servidor", "computador",
    "mensagem", "comunicação", "envio", "troca",
    "eleição", "líder", "coordenador", "escolhido",
    "anel", "círculo", "vizinho", "sucessor",
    "tempo", "passo", "estado", "evolução"
]

def calcular_score_negocio(texto):
    texto_lower = texto.lower()
    score = 0
    for termo in TERMOS_DE_NEGOCIO:
        if termo in texto_lower:
            score += 1
    return score

def calcular_penalidade_jargao(texto):
    texto_lower = texto.lower()
    score = 0
    for termo in JARGOES_TECNICOS:
        if termo in texto_lower:
            score += 1
    return score

# Função Flesch adaptada para Português (Martins & Forghieri)
def flesch_pt(texto):
    try:
        palavras = texto.split()
        num_palavras = len(palavras)
        if num_palavras == 0: return 0

        # Estimativa simples de sílabas (contando vogais como aproximação rápida)
        vogais = "aeiouáéíóúàèìòùâêîôûãõ"
        num_silabas = sum(1 for char in texto.lower() if char in vogais)
        num_frases = texto.count('.') + texto.count('!') + texto.count('?')
        if num_frases == 0: num_frases = 1

        # Fórmula Flesch Adaptada
        score = 248.835 - (1.015 * (num_palavras / num_frases)) - (84.6 * (num_silabas / num_palavras))
        return max(0, min(100, score)) # Trava entre 0 e 100
    except:
        return 0

resultados = []

print(f"Iniciando Benchmark em {len(df_alloy)} arquivos...")

for index, row in tqdm(df_alloy.iterrows(), total=df_alloy.shape[0]):
    arquivo = row['arquivo']
    codigo = row['codigo_alloy']

    # 1. GENERALISTA (GPT-4o Zero-Shot)
    start = time.time()
    resp_gen = generalist_model(codigo)
    tempo_gen = time.time() - start

    # 2. ESPECIALISTA (GPT-4o System Prompt)
    start = time.time()
    resp_esp = specialist_model(codigo)
    tempo_esp = time.time() - start

    # Coleta de Métricas
    resultados.append({
        "Arquivo": arquivo,

        # Generalista
        "Gen_Score_Negocio": calcular_score_negocio(resp_gen), # Quanto ele explicou o conceito?
        "Gen_Jargao": calcular_penalidade_jargao(resp_gen),    # Quanto ele usou código no meio do texto?
        "Gen_Leiturabilidade_PT": flesch_pt(resp_gen),         # Quão fácil é ler?

        # Especialista
        "Esp_Score_Negocio": calcular_score_negocio(resp_esp),
        "Esp_Jargao": calcular_penalidade_jargao(resp_esp),
        "Esp_Leiturabilidade_PT": flesch_pt(resp_esp),

        # Textos
        "Gen_Texto": resp_gen,
        "Esp_Texto": resp_esp
    })

    time.sleep(1) # Pausa para não bloquear a API

# Salvar e Mostrar
df_final = pd.DataFrame(resultados)
df_final.to_excel("Resultado_Final_Metricas.xlsx", index=False)

print("Análise concluída! Baixe o arquivo 'Resultado_Final_Metricas.xlsx'.")
display(df_final.head())

In [ ]:
!pip install -q textstat

import textstat
import pandas as pd

print("Calculando Complexidade de Leitura (Flesch Score)...")

resultados_leitura = []


for index, row in df_final.iterrows():
    texto_gen = row['Gen_Texto']
    texto_esp = row['Esp_Texto']

    score_gen = flesch_pt(texto_gen)
    score_esp = flesch_pt(texto_esp)

    resultados_leitura.append({
        "Arquivo": row['Arquivo'],
        "Gemini_Leiturabilidade": score_gen,
        "Especialista_Leiturabilidade": score_esp,
        "Vencedor_Clareza": "Generalista" if score_gen > score_esp else "Especialista"
    })

df_leitura = pd.DataFrame(resultados_leitura)

print("\nCálculo de Leiturabilidade Concluído!")
display(df_leitura)